<a href="https://colab.research.google.com/github/Mohan0332/Capstone/blob/main/basicCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image
import os

In [11]:
# function to convert a image to grayscale 
def rgb_to_gray(rgb_img):
    # Convert the image to a NumPy array
    rgb_array = np.array(rgb_img)

    # Create an empty grayscale image array with the same dimensions as the RGB image
    gray_array = np.empty(rgb_array.shape[:2], dtype=np.uint8)

    # Calculate the luminance value for each pixel in the RGB image and assign it to the grayscale image array
    for i in range(rgb_array.shape[0]):
        for j in range(rgb_array.shape[1]):
            r, g, b,al = rgb_array[i,j]
            luminance = int(0.2989 * r + 0.5870 * g + 0.1140 * b)
            gray_array[i,j] = luminance
    
    gray_img = Image.fromarray(gray_array)
    return gray_img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
data = []
labels = []
classes = 2
cur_path = '/content/drive/MyDrive/Capstone dataset/Images-out'

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize((100,100))
            #img = rgb_to_gray(image)
            image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

In [43]:
data = np.array(data)
labels = np.array(labels)

In [44]:
print(data.shape, labels.shape)

(490, 100, 100, 4) (490,)


In [45]:
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [46]:
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [47]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(392, 100, 100, 4) (98, 100, 100, 4) (392,) (98,)


In [29]:
'''X_train = np.expand_dims(X_train, axis=0)
X_test = np.expand_dims(X_test, axis=0)'''

In [48]:
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [49]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(2, activation='softmax'))


In [51]:
#Compilation of the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [52]:
epochs = 25
history = model.fit(X_train, y_train, batch_size=16, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/25
25/25 [==============================] - 39s 1s/step - loss: 5.7460 - accuracy: 0.6250 - val_loss: 0.6983 - val_accuracy: 0.5306
Epoch 2/25
25/25 [==============================] - 38s 1s/step - loss: 0.6823 - accuracy: 0.6173 - val_loss: 0.6928 - val_accuracy: 0.5306
Epoch 3/25
25/25 [==============================] - 39s 2s/step - loss: 0.6682 - accuracy: 0.6173 - val_loss: 0.7076 - val_accuracy: 0.5306
Epoch 4/25
25/25 [==============================] - 38s 2s/step - loss: 0.6694 - accuracy: 0.6173 - val_loss: 0.7137 - val_accuracy: 0.5306
Epoch 5/25
25/25 [==============================] - 38s 2s/step - loss: 0.6717 - accuracy: 0.6173 - val_loss: 0.7026 - val_accuracy: 0.5306
Epoch 6/25
25/25 [==============================] - 37s 1s/step - loss: 0.6720 - accuracy: 0.6173 - val_loss: 0.7025 - val_accuracy: 0.5306
Epoch 7/25
25/25 [==============================] - 40s 2s/step - loss: 0.6687 - accuracy: 0.6173 - val_loss: 0.7109 - val_accuracy: 0.5306
Epoch 8/25
25/25 [==

In [54]:
data = []
labels = []
classes = 2
cur_path = '/content/drive/MyDrive/Capstone dataset/Images-out'

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'test',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize((100,100))
            #img = rgb_to_gray(image)
            image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

In [55]:
X_test=np.array(data)

In [57]:
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

2/2 [==============================] - 1s 417ms/step


In [60]:
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, y_pred))

0.5964912280701754
